In [178]:
import numpy as np
from tqdm import tqdm
from hw2_code.kmeans import KMeans

# data = np.random.randn(5, 4)
# data[:, 1] = np.array((1,0.5,1,-1,0.641))
data = np.array([[0.10782656, 0.04982049, 0.78844897, 0.05390398],
                [0.16080479, 0.70138883, 0.05805256, 0.07975382],
                [0.39637071, 0.23624673, 0.0996817 , 0.26770086],
                [0.21741805, 0.56913777, 0.05890126, 0.15454293],
                [0.27040961, 0.54778227, 0.01886611, 0.16294201]])

# data[4] = np.zeros((1,4))

# data = np.resize(np.arange(20), (5,4))

print(data)

[[0.10782656 0.04982049 0.78844897 0.05390398]
 [0.16080479 0.70138883 0.05805256 0.07975382]
 [0.39637071 0.23624673 0.0996817  0.26770086]
 [0.21741805 0.56913777 0.05890126 0.15454293]
 [0.         0.         0.         0.        ]]


In [193]:
### Helper functions
"""
logit is N x D
"""
logit = data

## softmax
def softmax(logit):
    prob = np.exp(logit - np.max(logit, axis=-1, keepdims=True)) / np.sum(np.exp(logit - np.max(logit, axis=-1, keepdims=True)), axis=-1, keepdims=True)
    return prob

prob = softmax(logit)

## logsumexp
def logsumexp(logit):
    s = np.log(np.sum(np.exp(logit - np.max(logit, axis=-1, keepdims=True)), axis=-1, keepdims=True)) + np.max(logit, axis=-1, keepdims=True)
    return s

s = logsumexp(logit)

## multinormalPDF
"""
Args:
    points: N x D numpy array
    mu_i: (D,) numpy array, the center for the ith gaussian.
    sigma_i: DxD numpy array, the covariance matrix of the ith gaussian.
Return:
    normal_pdf: (N,) numpy array, the probability density value of N data for the ith gaussian

Hint:
    1. np.linalg.det() and np.linalg.inv() should be handy.
    2. The value in self.D may be outdated and not correspond to the current dataset,
    try using another method involving the current arguments to get the value of D
"""
SIGMA_CONST = 1e-6
mu_i = data.mean(axis=0)
sigma_i = np.cov(data, rowvar=False)
points = data

def multinormalPDF(points, mu_i, sigma_i):
    D = mu_i.shape[0]
    try:
        inv = np.linalg.inv(sigma_i)
    except np.linalg.LinAlgError:
        inv = np.linalg.inv(sigma_i + SIGMA_CONST)

    semi = (points - mu_i) @ inv
    # NN =  np.sum((-0.5 * (semi @ (semi.T * (points - mu_i).T))), axis=0) # TÔ CONFUSO SE É PRA SER 1XN OU NXN (LOCALTEST INDICA QUE É NXN)
    NN =  -0.5 * (semi @ (semi.T * (points - mu_i).T))
    normal_pdf = (1/((2*np.pi) ** (D/2))) * (np.linalg.det(sigma_i) ** (-0.5)) * np.exp(NN)
    return normal_pdf

mnpdf = multinormalPDF(points, mu_i, sigma_i)

# mnpdf.shape
print(prob)
print()
print(s)

[[0.20549678 0.19391585 0.40587806 0.19470932]
 [0.22019125 0.37807044 0.19868969 0.20304863]
 [0.28780478 0.24522065 0.21391813 0.25305644]
 [0.23730538 0.33733199 0.20251846 0.22284417]
 [0.25       0.25       0.25       0.25      ]]

[[1.69015149]
 [1.67406359]
 [1.64184359]
 [1.65582548]
 [1.38629436]]


C:\Users\arthu\AppData\Local\Temp\ipykernel_11976\3118478574.py:50: RuntimeWarning: overflow encountered in exp
  normal_pdf = (1/((2*np.pi) ** (D/2))) * (np.linalg.det(sigma_i) ** (-0.5)) * np.exp(NN)


In [174]:
### Initialization and log-likelihood

N = data.shape[0]
D = data.shape[1]
K = 3

## __init_components

def _init_components(K, D, N, data):

    np.random.seed(5) #Do Not Remove Seed

    pi = np.ones((5, 1)) / K
    mu = np.ones((K, D))
    for i in range(K):
        mu[i] = data[int(np.random.uniform(0, N-1))] # sei lá pq N-1, mas eu tentei do nadão e funcionou
    sigma = np.ones((K, D, D)) * np.eye(D)

    return pi, mu, sigma

pi, mu, sigma = _init_components(K, D, N, data)
# print(mu)
# print(mu.shape)

## _ll_joint